In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/train.csv')
                    
train.head()

In [ ]:
train['target']= [ 1 if i>= 0.5 else 0 for i in train['target']]
train.head()

In [ ]:
train.isnull().sum()

In [ ]:
X = train['comment_text']
y = train['target']

In [ ]:
import re
def preprocess(text):
    text = text.str.lower()
    text = text.str.replace(r'\r', ' ')
    text = text.str.replace(r'\n', ' ')
    text = text.str.replace('[^a-zA-Z0-9]', ' ')
    return text

In [ ]:
X = preprocess(X)

In [ ]:
y.value_counts()

In [ ]:

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

def stemi(docu):
    texts =  [SnowballStemmer.stem(t) for t in docu.split() if t not in stopwords]
    doc= ' '.join(texts)
    return doc
    

In [ ]:
stopwords = set(stopwords.words('english'))
SnowballStemmer = SnowballStemmer("english")


In [ ]:
x = X.apply(lambda docu : stemi(docu))

In [ ]:
x

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfid = TfidfVectorizer(min_df=0.01)

In [ ]:
x = tfid.fit_transform(x)


In [ ]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
YY = enc.fit_transform(y)

In [ ]:
import tensorflow as tf
import keras.backend as K
from keras import layers, models, optimizers, regularizers
from keras.models import Sequential

input_layer = layers.Input((x.shape[1], ), sparse=True)
hidden = layers.Dense(500, activation='relu')(input_layer)
drop= layers.Dropout(0.3)(hidden)
output_layer = layers.Dense(1, activation='sigmoid')(drop)


In [ ]:
model = models.Model(inputs= input_layer, outputs = output_layer)

In [ ]:
model.compile(optimizer= optimizers.Adam(lr=0.001, amsgrad=True), loss= 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
Es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
Mc = ModelCheckpoint('model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

In [ ]:
import scipy 
scipy.sparse.csr_matrix.sort_indices(x)


In [ ]:
x

In [ ]:
model.fit(x, YY, batch_size = 256, epochs = 10, callbacks=[Es, Mc], verbose=1  )


In [ ]:
test = pd.read_csv("/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test.csv")
test = preprocess(test['comment_text'])
test = test.apply(lambda docu : stemi(docu))
test = tfid.transform(test)


In [ ]:
scipy.sparse.csr_matrix.sort_indices(test)
pred = model.predict(test)
pred = np.where(pred>=0.5,1,0)


In [ ]:
sub_df = pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv')
sub_df['prediction'] = pred

In [ ]:
sub_df.to_csv('submission.csv',index = False)